In [1]:
import os
import pandas as pd
import numpy as np
import earthpy as et

In [3]:
curr_timespan = '2014to2023'
data_dir = os.path.join(et.io.HOME, 'data')
out_dir = os.path.join(data_dir, 'out')
nwcgdata_dir = os.path.join(data_dir,'raw','common_data')
tmp_dir = os.path.join(data_dir, 'tmp')

In [4]:
df = pd.read_csv(os.path.join(out_dir,
                                  'SIT209_HISTORY_INCIDENT_209_REPORTS_{}_cleaned.csv'.format(curr_timespan)), 
                     parse_dates=True,
                     low_memory=False)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
dfsub = df.loc[df.INCIDENT_ID.isin(['2021_12993824_DIXIE','2021_12908560_CALDOR'])][['INCIDENT_ID',
                                                                                   'INC209R_IDENTIFIER',
                                                                                   'REPORT_TO_DATE',
                                                                                   'CURRENT_THREAT_12',
                                                                                   'CURRENT_THREAT_24',
                                                                                   'CURRENT_THREAT_48',
                                                                                   'CURRENT_THREAT_72',
                                                                                   'CURRENT_THREAT_GT72',
                                                                                   'CURRENT_THREAT_NARR',
                                                                                   'STR_THREATENED_RES',
                                                                                   'STR_THREATENED',
                                                                                   'STR_DESTROYED',
                                                                    'LIFE_SAFETY_HEALTH_STATUS_NARR']].copy()
dfsub[['CURRENT_THREAT_12',
       'CURRENT_THREAT_24',
       'CURRENT_THREAT_48',
       'CURRENT_THREAT_72',
       'CURRENT_THREAT_GT72']] = dfsub[['CURRENT_THREAT_12',
                                        'CURRENT_THREAT_24',
                                        'CURRENT_THREAT_48',
                                        'CURRENT_THREAT_72',
                                        'CURRENT_THREAT_GT72']].fillna('')

In [6]:
dft = dfsub.copy()
collist = ['CURRENT_THREAT_12','CURRENT_THREAT_24','CURRENT_THREAT_48','CURRENT_THREAT_72','CURRENT_THREAT_GT72']
#collist = ['CURRENT_THREAT_12','CURRENT_THREAT_24']
for col in collist:
    dft[col] = df[col].str.strip()

# initialize appended to False and consolidated narrative to empty string
dft['CURRENT_THREAT_NARR2'] = ''
for col in collist:
    time_frame = col.split('_')[-1]
    # strip to make sure not null
    if time_frame == '12':
        # initialize processed content
        dft.loc[dft[col] != '','prev_narr'] = dft['CURRENT_THREAT_12']
        dft.loc[dft[col] != '','time_label'] = time_frame
    else: # later time_frame
        # equal to previous narrative -- add time to time label
        dft.loc[(dft[col] != '') & (dft[col].str.lower() == dft['prev_narr'].str.lower()),'time_label'] = dft.time_label + '/' + time_frame
        # not equal -- append and re-initialize
        dft.loc[((dft[col] != '') & (dft[col].str.lower() != dft['prev_narr'].str.lower())),'CURRENT_THREAT_NARR2'] = dft.CURRENT_THREAT_NARR2 + \
                                                                                  "\n" + dft.time_label + " Hours: " + \
                                                                                  dft.prev_narr
        dft.loc[(dft[col] != '') & (dft[col].str.lower() != dft['prev_narr'].str.lower()),'time_label'] = time_frame
        dft.loc[(dft[col] != '') & (dft[col].str.lower() != dft['prev_narr'].str.lower()),'prev_narr'] = dft[col]
# append remaining values if appended is False
dft['CURRENT_THREAT_NARR2'] = dft.CURRENT_THREAT_NARR2 + "\n" + \
                                                dft.time_label + " Hours: " + dft.prev_narr
dft['CURRENT_THREAT_NARR2'] = dft.CURRENT_THREAT_NARR2.str.strip()
        
#dft.drop(['prev_narr','time_label'],axis=1,inplace=True)
dft.to_csv(os.path.join(tmp_dir,'dixie-caldor-threat.csv'),index=False)